### Preliminary setup

In [ ]:
%pip install pyarrow

In [29]:
import pandas as pd             # For data transformation
import glob                     # For loading all Parquet files

from functools import reduce

### Reading Parquet files

In [25]:
bpi_parquet_files = glob.glob('base-datasets/BPI/*.parquet')
bpi_dfs = [pd.read_parquet(data) for data in bpi_parquet_files]

In [ ]:
for x in bpi_dfs:
    print(x.head())

In [28]:
print([x.shape for x in bpi_dfs])

[(143165, 14), (92204, 18), (35718, 10), (522409, 12), (100000, 3)]


From the SEGMENTS_Q42023_Q12024 file, we filter so that the entries are only BUSINESS BANKING customers (MSMEs).

In [51]:
bpi_dfs[4].shape # dataframe of SEGMENTS_Q42023_Q12024.parquet

(100000, 3)

In [52]:
segments_filtered = bpi_dfs[4][bpi_dfs[4]['CUSTOMER_GROUP'] == 'BUSINESS BANKING']
segments_filtered.shape

(10000, 3)

From this, we expect that we have at most 10000 unique CUSTOMER_ID values that may be found across the other parquet files that we can merge into the master dataset.

### CCCONSO Dataset

In [54]:
bpi_dfs[0].shape # dataframe of CCCONSO

(143165, 14)

In [55]:
ccconso_filtered = bpi_dfs[0][bpi_dfs[0]['CUSTOMER_ID'].isin(segments_filtered['CUSTOMER_ID'])]
ccconso_filtered.shape # reduced CCCONSO dataset

(16338, 14)

### CDNA Dataset

In [56]:
bpi_dfs[1].shape # dataframe of CDNA

(92204, 18)

In [57]:
cdna_filtered = bpi_dfs[1][bpi_dfs[1]['CUSTOMER_ID'].isin(segments_filtered['CUSTOMER_ID'])]
cdna_filtered.shape # reduced CDNA dataset

(2204, 18)

### LOAN Dataset

In [58]:
bpi_dfs[2].shape # dataframe of LOAN

(35718, 10)

In [59]:
loan_filtered = bpi_dfs[2][bpi_dfs[2]['CUSTOMER_ID'].isin(segments_filtered['CUSTOMER_ID'])]
loan_filtered.shape # reduced LOAN dataset

(13782, 10)

### PRODUCTS Dataset

In [60]:
bpi_dfs[3].shape # dataframe of PRODUCTS

(522409, 12)

In [61]:
products_filtered = bpi_dfs[3][bpi_dfs[3]['CUSTOMER_ID'].isin(segments_filtered['CUSTOMER_ID'])]
products_filtered.shape # reduced PRODUCTS dataset

(41548, 12)

### Merging

In [ ]:
# bpi_merged = reduce(lambda left, right: pd.merge(left, right, on='CUSTOMER_ID', how='inner'), bpi_dfs)
# bpi_merged